<a href="https://colab.research.google.com/github/DinhNgocVan/NLP_XLNet_Final_Project/blob/main/imdb_XLNetBaseModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# XLNet IMDB movie review use XLNet base model

This is so bad that in colab do not has enough resources to use large model of XLNet. So that, I cannot compare the error rate of the IMDB like what paper show because they just show the result of large model. 
Thus, I implement their code with `xlnet-base-model` to compare with my own implementation.

This notebook is for classifying the [imdb sentiment dataset](https://ai.stanford.edu/~amaas/data/sentiment/). 

## Setup
Install dependencies

In [ ]:
! pip install sentencepiece

     |████████████████████████████████| 1.2 MB 7.7 MB/s 


Download the pretrained XLNet model and unzip

In [ ]:
# only needs to be done once
! wget https://storage.googleapis.com/xlnet/released_models/cased_L-12_H-768_A-12.zip
! unzip cased_L-12_H-768_A-12.zip

--2022-01-14 05:20:24--  https://storage.googleapis.com/xlnet/released_models/cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.135.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 433638019 (414M) [application/zip]
Saving to: ‘cased_L-12_H-768_A-12.zip’

cased_L-12_H-768_A- 100%[===================>] 413.55M   187MB/s    in 2.2s    

2022-01-14 05:20:27 (187 MB/s) - ‘cased_L-12_H-768_A-12.zip’ saved [433638019/433638019]

Archive:  cased_L-12_H-768_A-12.zip
   creating: xlnet_cased_L-12_H-768_A-12/
  inflating: xlnet_cased_L-12_H-768_A-12/xlnet_model.ckpt.index  
  inflating: xlnet_cased_L-12_H-768_A-12/xlnet_model.ckpt.data-00000-of-00001  
  inflating: xlnet_cased_L-12_H-768_A-12/spiece.model  
  inflating: xlnet_cased_L-12_H-768_A-12/xlnet_model.ckpt.meta  
  inflating: xlnet_cased_L-12_H-768_A-12/xl

Download extract the imdb dataset - surpessing output

In [ ]:
! wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
! tar zxf aclImdb_v1.tar.gz

--2022-01-14 05:20:49--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  59.5MB/s    in 1.3s    

2022-01-14 05:20:50 (59.5 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



Git clone XLNet repo for access to run_classifier and the rest of the xlnet module

In [ ]:
! git clone https://github.com/zihangdai/xlnet.git

Cloning into 'xlnet'...
remote: Enumerating objects: 122, done.
remote: Total 122 (delta 0), reused 0 (delta 0), pack-reused 122
Receiving objects: 100% (122/122), 2.92 MiB | 32.51 MiB/s, done.
Resolving deltas: 100% (59/59), done.


## Define Variables
Define all the dirs: data, xlnet scripts & pretrained model. 
If you would like to save models then you can authenticate a GCP account and use that for the OUTPUT_DIR & CHECKPOINT_DIR - you will need a large amount storage to fix these models. 

Alternatively it is easy to integrate a google drive account, checkout this guide for [I/O in colab](https://colab.research.google.com/notebooks/io.ipynb) but rememeber these will take up a large amount of storage. 


In [ ]:
SCRIPTS_DIR = 'xlnet' #@param {type:"string"}
DATA_DIR = 'aclImdb' #@param {type:"string"}
OUTPUT_DIR = 'proc_data/imdb' #@param {type:"string"}
PRETRAINED_MODEL_DIR = 'xlnet_cased_L-12_H-768_A-12' #@param {type:"string"}
CHECKPOINT_DIR = 'exp/imdb' #@param {type:"string"}

## Run Model
This will set off the fine tuning of XLNet. There are a few things to note here:


1.   This script will train and evaluate the model
2.   This will store the results locally on colab and will be lost when you are disconnected from the runtime
3.   This uses the large version of the model (base not released presently)
4.   We are using a max seq length of 128 with a batch size of 8 please refer to the [README](https://github.com/zihangdai/xlnet#memory-issue-during-finetuning) for why this is.
5. This will take approx 4hrs to run on GPU.



In [ ]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3 MB 27 kB/s 
     |████████████████████████████████| 3.8 MB 69.2 MB/s 
     |████████████████████████████████| 503 kB 76.0 MB/s 
     |████████████████████████████████| 50 kB 8.4 MB/s 
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7554 sha256=e1f1d9459bc36a37b3853634679ed859f71e861abef553e3efb60dcf8d7ee784
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
Successfully built gast
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.7.0
    Uninstalling tensorflow-estimator-2.7.0:
      Successfully uninstalled tensorflow-estimator-2.7.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.7.0
    Uninstalling tensorboard-2.7.0:
      Successfully uninstalled tensorboard-2.7.0
  Attempting uninstall: gast
    Found existing installation: gast 0.4.0
    Uninstalling gast-0.4.0:
      Succes

In [ ]:
train_command = "python xlnet/run_classifier.py \
  --do_train=True \
  --do_eval=True \
  --eval_all_ckpt=True \
  --task_name=imdb \
  --data_dir="+DATA_DIR+" \
  --output_dir="+OUTPUT_DIR+" \
  --model_dir="+CHECKPOINT_DIR+" \
  --uncased=False \
  --spiece_model_file="+PRETRAINED_MODEL_DIR+"/spiece.model \
  --model_config_path="+PRETRAINED_MODEL_DIR+"/xlnet_config.json \
  --init_checkpoint="+PRETRAINED_MODEL_DIR+"/xlnet_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=32 \
  --eval_batch_size=32 \
  --num_hosts=1 \
  --num_core_per_host=1 \
  --learning_rate=2e-5 \
  --train_steps=400 \
  --warmup_steps=250 \
  --save_steps=500 \
  --iterations=500"

! {train_command}





W0114 05:29:18.865553 140457405339520 module_wrapper.py:139] From xlnet/run_classifier.py:637: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0114 05:29:18.865727 140457405339520 module_wrapper.py:139] From xlnet/run_classifier.py:637: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0114 05:29:18.865899 140457405339520 module_wrapper.py:139] From xlnet/run_classifier.py:661: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.


W0114 05:29:18.911862 140457405339520 module_wrapper.py:139] From /content/xlnet/model_utils.py:27: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.


W0114 05:29:18.912081 140457405339520 module_wrapper.py:139] From /content/xlnet/model_utils.py:36: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

INFO:tensorflow:Single device mode.
I0114 05:29:18.912180 140457

## Running & Results
These are the results that I got from running this experiment
### Params
*    --max_seq_length=128 \
     --train_batch_size=32 \
     --eval_batch_size=32 \
     --train_steps=400 \
     --warmup_steps=250 \
In my own implementation I put the number of epochs = 4, and batch_size = 32 so that I have to change value to compare with my implementation.

Besides, I caculate the train steps and warmup steps with the formula below:

```
train_steps = int(len(train_dataset) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * 0.1)
```

### Results
*  Most accurate model on final step
*   **Accuracy: 0.895359992980957, error rate = (1 - acccuracy)*100 = 10.464**


### Model

*   The trained model checkpoints can be found in 'exp/imdb'

